In [1]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup
import string
import spacy
import re

In [2]:
url = """https://insights.blackcoffer.com/how-machine-learning-will-affect-your-business/"""

In [3]:
headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0"}
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
soup=BeautifulSoup(page.content, 'html.parser')

In [5]:
title=soup.find('h1',class_="entry-title")
title=title.text.replace('\n'," ")
title

'How machine learning will affect your business?'

In [6]:
content=soup.findAll(attrs={'class':'td-post-content'})
content=content[0].text.replace('\n'," ")
content

' Machine learning techniques may have been used for years, but recently there has been an explosion in their applications. In fact, in a recent Q3 earnings call, Google CEO Sundar Pichai said “Machine learning is a core, transformative way by which we’re re-thinking how we’re doing everything.” And they’re far from the only business making that claim. In the past, successful use of machine learning algorithms required bespoke algorithms and huge R&D budgets, but all that is changing. IBM Watson, Microsoft Azure, Amazon, and Alibaba all launched turnkey cloud-based machine-learning SaaS solutions in 2015. At the same time startups like Idibon, MetaMind, Dato, and MonkeyLearn have built machine learning products that companies can take advantage of. Gartner already puts machine learning at the top of its hype curve, and no: machine learning won’t replace all of your employees with computers or suddenly double your revenue. But that doesn’t mean that it can’t give every business a compet

In [7]:
#Punctuation
content = content.translate(str.maketrans('', '', string.punctuation)) 
content

' Machine learning techniques may have been used for years but recently there has been an explosion in their applications In fact in a recent Q3 earnings call Google CEO Sundar Pichai said “Machine learning is a core transformative way by which we’re rethinking how we’re doing everything” And they’re far from the only business making that claim In the past successful use of machine learning algorithms required bespoke algorithms and huge RD budgets but all that is changing IBM Watson Microsoft Azure Amazon and Alibaba all launched turnkey cloudbased machinelearning SaaS solutions in 2015 At the same time startups like Idibon MetaMind Dato and MonkeyLearn have built machine learning products that companies can take advantage of Gartner already puts machine learning at the top of its hype curve and no machine learning won’t replace all of your employees with computers or suddenly double your revenue But that doesn’t mean that it can’t give every business a competitive advantage There are

In [8]:
text = content.split()
text

['Machine',
 'learning',
 'techniques',
 'may',
 'have',
 'been',
 'used',
 'for',
 'years',
 'but',
 'recently',
 'there',
 'has',
 'been',
 'an',
 'explosion',
 'in',
 'their',
 'applications',
 'In',
 'fact',
 'in',
 'a',
 'recent',
 'Q3',
 'earnings',
 'call',
 'Google',
 'CEO',
 'Sundar',
 'Pichai',
 'said',
 '“Machine',
 'learning',
 'is',
 'a',
 'core',
 'transformative',
 'way',
 'by',
 'which',
 'we’re',
 'rethinking',
 'how',
 'we’re',
 'doing',
 'everything”',
 'And',
 'they’re',
 'far',
 'from',
 'the',
 'only',
 'business',
 'making',
 'that',
 'claim',
 'In',
 'the',
 'past',
 'successful',
 'use',
 'of',
 'machine',
 'learning',
 'algorithms',
 'required',
 'bespoke',
 'algorithms',
 'and',
 'huge',
 'RD',
 'budgets',
 'but',
 'all',
 'that',
 'is',
 'changing',
 'IBM',
 'Watson',
 'Microsoft',
 'Azure',
 'Amazon',
 'and',
 'Alibaba',
 'all',
 'launched',
 'turnkey',
 'cloudbased',
 'machinelearning',
 'SaaS',
 'solutions',
 'in',
 '2015',
 'At',
 'the',
 'same',
 'time'

In [9]:
len(text)

726

In [11]:
#Positive Score 
with open(r"C:\Users\Om Bhandwalkar\Desktop\pos\positive-words.txt") as pos:
    poswords = pos.read().split("\n")  
    poswords = poswords[5:]
pos_count = " ".join ([w for w in text if w in poswords])
pos_count=pos_count.split(" ")
Positive_score=len(pos_count)
print(Positive_score)

42


In [12]:
#Negative Score
with open(r"C:\Users\Om Bhandwalkar\Desktop\pos\negative-words.txt",encoding ="ISO-8859-1") as neg:
    negwords = neg.read().split("\n")
    
negwords = negwords[36:]
neg_count = " ".join ([w for w in text if w in negwords])
neg_count=neg_count.split(" ")
Negative_score=len(neg_count)
print(Negative_score)

13


In [13]:
filter_content = ' '.join(text)

In [14]:
data=[[url,title,content,filter_content,Positive_score,Negative_score]]

In [15]:
data=pd.DataFrame(data,columns=["url","title","content","filter_content","Positive_Score","Negative_Score"])

In [16]:
from textblob import TextBlob

In [17]:
# Get The Subjectivity
def sentiment_analysis(data):
    sentiment = TextBlob(data["content"]).sentiment
    return pd.Series([sentiment.polarity,sentiment.subjectivity ])

# Adding Subjectivity & Polarity
data[["polarity", "subjectivity"]] = data.apply(sentiment_analysis, axis=1)

data

,url,title,content,filter_content,Positive_Score,Negative_Score,polarity,subjectivity
0,https://insights.blackcoffer.com/how-machine-l...,How machine learning will affect your business?,Machine learning techniques may have been use...,Machine learning techniques may have been used...,42,13,0.190177,0.566437


In [18]:
#AVG SENTENCE LENGTH
AVG_SENTENCE_LENGTH = len(content.replace(' ',''))/len(re.split(r'[?!.]', content))
print('Word average =', AVG_SENTENCE_LENGTH)

Word average = 3521.0


In [19]:
import textstat

In [20]:
#Fog index 
FOG_INDEX=(textstat.gunning_fog(content))
print(FOG_INDEX)

293.6


In [21]:
#Average No of Words Per Sentence 
AVG_NUMBER_OF_WORDS_PER_SENTENCE = [len(l.split()) for l in re.split(r'[?!.]', content) if l.strip()]
AVG_NUMBER_OF_WORDS_PER_SENTENCE=print(sum(AVG_NUMBER_OF_WORDS_PER_SENTENCE)/len(AVG_NUMBER_OF_WORDS_PER_SENTENCE))

726.0


In [22]:
#Complex words
def syllable_count(word):
    count = 0
    vowels = "AEIOUYaeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)): 
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
            if word.endswith("es"or "ed"):
                count -= 1
    if count == 0:
        count += 1
    return count



COMPLEX_WORDS=syllable_count(content)
print(COMPLEX_WORDS)

1230


In [23]:
#Word Count
Word_Count=len(content)
print(Word_Count)

4248


In [24]:
#Percentage Complex Words
pcw=(COMPLEX_WORDS/Word_Count)*100
print(pcw)

28.954802259887007


In [25]:
#Average Word Length
Average_Word_Length=len(content.replace(' ',''))/len(content.split())
print(Average_Word_Length)

4.849862258953168


In [26]:
#Syllable Count Per Word
word=content.replace(' ','')
syllable_count=0
for w in word:
      if(w=='a' or w=='e' or w=='i' or w=='o' or w=='y' or w=='u' or w=='A' or w=='E' or w=='I' or w=='O' or w=='U' or w=='Y'):
            syllable_count=syllable_count+1
print("The AVG number of syllables in the word is: ")
print(syllable_count/len(content.split()))

The AVG number of syllables in the word is: 
1.943526170798898


In [2]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup
import string
import spacy
import re
url = """https://insights.blackcoffer.com/how-machine-learning-will-affect-your-business/"""
headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0"}
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')
soup=BeautifulSoup(page.content, 'html.parser')
title=soup.find('h1',class_="entry-title")
title=title.text.replace('\n'," ")
# title

content=soup.findAll(attrs={'class':'td-post-content'})
content=content[0].text.replace('\n'," ")
# print(content)
#Punctuation
content = content.translate(str.maketrans('', '', string.punctuation)) 
# print(content)
text = content.split()
# print(text)
len(text)
#Positive Score 
with open(r"C:\Users\Om Bhandwalkar\Desktop\pos\positive-words.txt") as pos:
    poswords = pos.read().split("\n")  
    poswords = poswords[5:]
pos_count = " ".join ([w for w in text if w in poswords])
pos_count=pos_count.split(" ")
Positive_score=len(pos_count)
print(Positive_score)

#Negative Score
with open(r"C:\Users\Om Bhandwalkar\Desktop\pos\negative-words.txt",encoding ="ISO-8859-1") as neg:
    negwords = neg.read().split("\n")
    
negwords = negwords[36:]
neg_count = " ".join ([w for w in text if w in negwords])
neg_count=neg_count.split(" ")
Negative_score=len(neg_count)
print(Negative_score)


filter_content = ' '.join(text)
data=[[url,title,content,filter_content,Positive_score,Negative_score]]
data=pd.DataFrame(data,columns=["url","title","content","filter_content","Positive_Score","Negative_Score"])
from textblob import TextBlob
# Get The Subjectivity
def sentiment_analysis(data):
    sentiment = TextBlob(data["content"]).sentiment
    return pd.Series([sentiment.polarity,sentiment.subjectivity ])
data[["polarity", "subjectivity"]] = data.apply(sentiment_analysis, axis=1)
data
#AVG SENTENCE LENGTH
AVG_SENTENCE_LENGTH = len(content.replace(' ',''))/len(re.split(r'[?!.]', content))
print('Word average =', AVG_SENTENCE_LENGTH)
import textstat
#Fog index 
FOG_INDEX=(textstat.gunning_fog(content))
print("FOG INDEX = ",FOG_INDEX)
#Average No of Words Per Sentence 
AVG_NUMBER_OF_WORDS_PER_SENTENCE = [len(l.split()) for l in re.split(r'[?!.]', content) if l.strip()]
print("Average no of words per sentence")
AVG_NUMBER_OF_WORDS_PER_SENTENCE=(sum(AVG_NUMBER_OF_WORDS_PER_SENTENCE)/len(AVG_NUMBER_OF_WORDS_PER_SENTENCE))
print(AVG_NUMBER_OF_WORDS_PER_SENTENCE)
#Complex words
def syllable_count(word):
    count = 0
    vowels = "AEIOUYaeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)): 
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
            if word.endswith("es"or "ed"):
                count -= 1
    if count == 0:
        count += 1
    return count



COMPLEX_WORDS=syllable_count(content)
print("Complex Words",COMPLEX_WORDS)
#Word Count
Word_Count=len(content)
print("Word Count",Word_Count)
#Percentage Complex Words
pcw=(COMPLEX_WORDS/Word_Count)*100
print("Percentage of Complex Words",pcw)
#Average Word Length
Average_Word_Length=len(content.replace(' ',''))/len(content.split())
print("Average Word per Length",Average_Word_Length)
#Syllable Count Per Word
word=content.replace(' ','')
syllable_count=0
for w in word:
      if(w=='a' or w=='e' or w=='i' or w=='o' or w=='y' or w=='u' or w=='A' or w=='E' or w=='I' or w=='O' or w=='U' or w=='Y'):
            syllable_count=syllable_count+1
print("The AVG number of syllables in the word is: ")
syllable = (syllable_count/len(content.split()))
print(syllable)

data = [[url,title,content,filter_content,Positive_score,Negative_score,AVG_SENTENCE_LENGTH,pcw,FOG_INDEX,
         AVG_NUMBER_OF_WORDS_PER_SENTENCE,COMPLEX_WORDS,Word_Count,syllable,Average_Word_Length]]
data=pd.DataFrame(data,columns=["url","title","content","filter_content","Positive_Score","Negative_Score","Avg_Sentence_Length"
                               ,"Percentage_Complex_Word","Fog_Index"," AVG_NUMBER_OF_WORDS_PER_SENTENCE","COMPLEX_WORDS",
                               "Word_Count","syllable","Average_Word_Length"])
from textblob import TextBlob
# Get The Subjectivity
def sentiment_analysis(data):
    sentiment = TextBlob(data["content"]).sentiment
    return pd.Series([sentiment.polarity,sentiment.subjectivity ])
data[["polarity", "subjectivity"]] = data.apply(sentiment_analysis, axis=1)
data



42
13
Word average = 3521.0
FOG INDEX =  293.6
Average no of words per sentence
726.0
Complex Words 1230
Word Count 4248
Percentage of Complex Words 28.954802259887007
Average Word per Length 4.849862258953168
The AVG number of syllables in the word is: 
1.943526170798898


,url,title,content,filter_content,Positive_Score,Negative_Score,Avg_Sentence_Length,Percentage_Complex_Word,Fog_Index,AVG_NUMBER_OF_WORDS_PER_SENTENCE,COMPLEX_WORDS,Word_Count,syllable,Average_Word_Length,polarity,subjectivity
0,https://insights.blackcoffer.com/how-machine-l...,How machine learning will affect your business?,Machine learning techniques may have been use...,Machine learning techniques may have been used...,42,13,3521.0,28.954802,293.6,726.0,1230,4248,1.943526,4.849862,0.190177,0.566437


In [3]:
data.to_csv(r'C:\Users\Om Bhandwalkar\Desktop\BlackCoffer Assignment\Output\url_45.csv')